# **Model Server**

_______________________________________________________________________________

<a id="top"></a>
## **steps**
**[notebook installs](#installs)**<br>
**[nuclio code section](#nuclio)**<br>
    - [inference server](#server)<br>
**[deploy](#deploy)**<br>
**[test deployment](#test)**<br>
**[test saved model object](#testingoutside)**


<a id="installs"></a>
_______________________________________________________________________________

## **notebook installs**

These are the packages we'll need to run this notebook, please install them once:

    !pip install -U kfserving==0.2.0 tensorflow==2.0.0b1 keras pandas 
    !pip install -U kubernetes==9.0.0
    !pip install -U azure

    !pip install -U git+https://github.com/yjb-ds/functions-demo.git
    !pip install -U git+https://github.com/mlrun/mlrun.git@development

<a id="nuclio"></a>
_______________________________________________________________________________

## **nuclio code section**

If you have never run nuclio functions in your notebooks, please uncomment and run the following:

    !pip install nuclio-jupyter

In [1]:
# nuclio: ignore
import nuclio

Install the following packages so they are available to the function:

In [2]:
%%nuclio cmd -c
pip install kfserving==0.2.0
pip install  numpy==1.16.4 tensorflow==2.0.0b1 pandas==0.25.3
pip install -U azure joblib
pip install -U git+https://github.com/yjb-ds/functions-demo.git
pip install -U git+https://github.com/mlrun/mlrun.git@development

In [3]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import kfserving
import numpy as np
import joblib
from typing import List
from tensorflow.keras.models import Sequential

from functions.models import FeaturesEngineer, classifier_gen
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

class MyKerasClassifier(kfserving.KFModel):
    def __init__(self,
                 name: str,
                 model_dir: str,
                 classifier: Sequential = None):
        """TFKerasClassifier
        
        KubeFlow serving model wrapper.
        
        :param name:            model name
        :param model_dir: path of stored model
        :param classifier:      class type of classifier model
        
        """
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if classifier:
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        ffg = FeaturesEngineer()

        # scaler
        ss = StandardScaler()
        ss.__dict__ = joblib.load(f"{self.model_dir}/scaler.pickle")

        # keras model
        ksm = classifier_gen()
        ksm.load_weights(f"{self.model_dir}/weights.h5")

        pipe = make_pipeline(ffg, ss, ksm)

        self.classifier = pipe

    def predict(self, body: List) -> List:
        """Generate model predictions from sample.
        
        :param body: A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [4]:
# nuclio: end-code

<a id="deploy"></a>
_______________________________________________________________________________

## **deploy**

In [5]:
from mlrun import new_model_server, mount_v3io

In [6]:
TARGET_PATH = "/User/mlrun/simdata"
MODEL_NAME = "tfkeras_serving_v2"

In [7]:
fn = new_model_server(MODEL_NAME, 
                      models={"tfkeras_joblib_v2": TARGET_PATH}, 
                      model_class="MyKerasClassifier").apply(mount_v3io())

In [8]:
#fn.verbose=True
addr = fn.deploy(project="refactoring-demos")

[mlrun] 2020-01-01 19:11:09,207 deploy started
[nuclio] 2020-01-01 19:11:09,276 (info) Building processor image
[nuclio] 2020-01-01 19:11:13,315 (info) Build complete
[nuclio] 2020-01-01 19:11:20,468 done updating tfkeras-serving-v2, function address: 3.137.70.243:30350


<a id="testingoutside"></a>
### **testing our model outside the server**

In [14]:
from sklearn.metrics  import accuracy_score

# create and load
model = MyKerasClassifier('anyname', model_dir=TARGET_PATH)
model.load()

model.predict({'instances' : features.values.tolist()})

(100, 20)
here


[[0.2862316370010376],
 [0.784155011177063],
 [0.9990143775939941],
 [0.03990083932876587],
 [0.8113516569137573],
 [0.9939308166503906],
 [0.09553894400596619],
 [0.7177563905715942],
 [0.9736238121986389],
 [0.9602663516998291],
 [0.8558810949325562],
 [0.01912662386894226],
 [0.7599364519119263],
 [0.08660963177680969],
 [0.9928114414215088],
 [0.028684556484222412],
 [0.010364711284637451],
 [0.014988690614700317],
 [0.01085364818572998],
 [0.14172640442848206],
 [0.8549386262893677],
 [0.10782825946807861],
 [0.8087799549102783],
 [0.02626052498817444],
 [0.020626991987228394],
 [0.8675011396408081],
 [0.034037262201309204],
 [0.7863306999206543],
 [0.04210638999938965],
 [0.9636402130126953],
 [0.014158755540847778],
 [0.014376461505889893],
 [0.9978076219558716],
 [0.9215453267097473],
 [0.008119255304336548],
 [0.002589106559753418],
 [0.6740338206291199],
 [0.9730592370033264],
 [0.0462263822555542],
 [0.8003339171409607],
 [0.5833727121353149],
 [0.24764549732208252],
 [0.993

## **test**

In [15]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

import requests

In [16]:
# Grab some data - balanced dataset
features = pd.read_csv("x_test_50.csv")
labels = pd.read_csv("y_test_50.csv")

In [17]:
event = {'instances' : features.values.tolist()}

In [18]:
resp = requests.put(addr + "/tfkeras_joblib_v2/predict", json=event)
print(resp.content)

b'Exception caught in handler "Failed to predict not enough values to unpack (expected 2, got 1)": Traceback (most recent call last):\n  File "/opt/nuclio/model_server.py", line 60, in predict\n    result: np.ndarray = self.classifier.predict(feats)\n  File "/usr/local/lib/python3.6/site-packages/sklearn/utils/metaestimators.py", line 116, in <lambda>\n    out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)\n  File "/usr/local/lib/python3.6/site-packages/sklearn/pipeline.py", line 417, in predict\n    Xt = transform.transform(Xt)\n  File "/usr/local/lib/python3.6/site-packages/functions/models.py", line 119, in transform\n    n, f = x.shape\nValueError: not enough values to unpack (expected 2, got 1)\n\nDuring handling of the above exception, another exception occurred:\n\nTraceback (most recent call last):\n  File "/opt/nuclio/_nuclio_wrapper.py", line 176, in serve_requests\n    entrypoint_output = self._entrypoint(self._context, event)\n  File "/opt/nuclio/model_server.py", 